In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as nnFn
import torch.optim as optim
import numpy as np
import random
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch_geometric.data import Data
from torch_geometric.nn import ARMAConv
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, log_loss
)
from torch.utils.data import TensorDataset, DataLoader, Subset
from torch_geometric.data import Data
from torch_geometric.nn import ARMAConv
from torch.utils.data import TensorDataset, DataLoader, Subset
from torchvision import models

In [2]:
data = np.load('pneumoniamnist_224.npz', allow_pickle=True)
all_images = np.concatenate([data['train_images'], data['val_images'], data['test_images']], axis=0)
all_labels = np.concatenate([data['train_labels'], data['val_labels'], data['test_labels']], axis=0).squeeze()

# Convert to 3-channel RGB
images = all_images.astype(np.float32) / 255.0
images = np.repeat(images[:, None, :, :], 3, axis=1)  # (N, 3, 224, 224)
X_torch = torch.tensor(images)
y_torch = torch.tensor(all_labels).long()
print(f"Raw images: {X_torch.shape}, Labels: {y_torch.shape}")

Raw images: torch.Size([5856, 3, 224, 224]), Labels: torch.Size([5856])


In [3]:
import torch
print("CUDA available:", torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU Name: NVIDIA A100 80GB PCIe


In [4]:
class0_idx = [i for i in range(len(y_torch)) if y_torch[i] == 0]
class1_idx = [i for i in range(len(y_torch)) if y_torch[i] == 1]

random.seed(42)
sampled_class0 = random.sample(class0_idx, min(2000, len(class0_idx)))
sampled_class1 = random.sample(class1_idx, min(2000, len(class1_idx)))

selected_indices = sampled_class0 + sampled_class1
random.shuffle(selected_indices)

subset_dataset = Subset(TensorDataset(X_torch, y_torch), selected_indices)
subset_loader = DataLoader(subset_dataset, batch_size=32, shuffle=False)

In [5]:
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()
resnet = resnet.to(device)
resnet.eval()

resnet_feats = []
y_list = []
with torch.no_grad():
    for imgs, labels in subset_loader:
        imgs = imgs.to(device)
        feats = resnet(imgs)
        resnet_feats.append(feats.cpu())
        y_list.extend(labels.cpu().tolist())

features = torch.cat(resnet_feats, dim=0).numpy().astype(np.float32)  # shape (N, feat_dim)
y_labels = np.array(y_list).astype(np.float32)
print("Feature shape:", features.shape, "Label shape:", y_labels.shape)

/data/home/va797/tmp_pyg118/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/data/home/va797/tmp_pyg118/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Feature shape: (3583, 512) Label shape: (3583,)


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as nnFn
from torch_geometric.nn import ARMAConv

class ARMA_Supervised(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device, num_stacks=1, num_layers=1):
        super().__init__()
        self.device = device

        # Only one ARMAConv layer
        self.conv1 = ARMAConv(input_dim, hidden_dim,
                              num_stacks=num_stacks, num_layers=num_layers,
                              shared_weights=True, dropout=0.2)

        self.dropout = nn.Dropout(0.25)
        self.fc = nn.Linear(hidden_dim, output_dim)

        # Activation function
        activations = {
            "SELU": nnFn.selu,
            "SiLU": nnFn.silu,
            "GELU": nnFn.gelu,
            "RELU": nnFn.relu,
            "ELU": nnFn.elu,
            "PReLU": nnFn.prelu,
            "LeakyReLU": nnFn.leaky_relu
        }
        self.act = activations.get("RELU", nnFn.relu)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Single layer
        x = self.conv1(x, edge_index)
        x = self.act(x)
        x = self.dropout(x)

        logits = self.fc(x)
        return logits

    def modularity_loss(self, A, logits):
        S = nnFn.softmax(logits, dim=1)
        d = torch.sum(A, dim=1)
        m = torch.sum(A)
        B = A - torch.outer(d, d) / (2 * m)

        modularity_term = (-1 / (2 * m)) * torch.trace(S.T @ B @ S)

        I_S = torch.eye(S.shape[1], device=self.device)
        k = torch.norm(I_S)
        n = S.shape[0]
        collapse_reg = (torch.sqrt(k) / n) * torch.norm(torch.sum(S, dim=0), p='fro') - 1
        entropy_reg = -torch.mean(torch.sum(S * torch.log(S + 1e-9), dim=1))

        return modularity_term + 0.1 * collapse_reg + 0.01 * entropy_reg


In [7]:
def create_adj(F, alpha=1):
    F_norm = F / np.linalg.norm(F, axis=1, keepdims=True)
    W = np.dot(F_norm, F_norm.T)
    W = np.where(W >= alpha, 1, 0).astype(np.float32)
    W = W / W.max()
    return W

def load_data(adj, node_feats):
    node_feats = torch.from_numpy(node_feats).float()
    edge_index = torch.from_numpy(np.array(np.nonzero((adj > 0))))
    return Data(x=node_feats, edge_index=edge_index)

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
alpha = 0.9
feats_dim = features.shape[1]
hidden_dim = 256
num_classes = 2
num_epochs = 2000
lr = 0.0001
weight_decay = 1e-4
batch_print_freq = 500
lambda_mod =  0.001  #0.2 #0.001  # weight for modularity loss
# lambda_sup = 1.0

In [9]:
W = create_adj(features, alpha)
data = load_data(W, features).to(device)
print(data)

Data(x=[3583, 512], edge_index=[2, 1642037])


In [10]:
sss = StratifiedShuffleSplit(n_splits=20, test_size=0.9, random_state=42)

accuracies, precisions, recalls, f1_scores, aucs, ce_losses = [], [], [], [], [], []

for fold, (train_idx, test_idx) in enumerate(sss.split(features, y_labels.astype(np.int64)), start=1):
    print(f"\n=== Fold {fold} ===")

    train_idx_t = torch.from_numpy(train_idx).long().to(device)
    y_train_tensor = torch.from_numpy(y_labels[train_idx]).long().to(device)

    model = ARMA_Supervised(feats_dim, hidden_dim, num_classes, device).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    ce_loss = nn.CrossEntropyLoss()

    for epoch in range(1, num_epochs + 1):
        model.train()
        optimizer.zero_grad()

        logits = model(data)
        loss = ce_loss(logits[train_idx_t], y_train_tensor)

        loss.backward()
        optimizer.step()

        if epoch % batch_print_freq == 0 or epoch == 1:
            model.eval()
            with torch.no_grad():
                preds_train = logits[train_idx_t].argmax(dim=1)
                acc_train = accuracy_score(y_train_tensor.cpu(), preds_train.cpu())
            print(f"Fold {fold} Epoch {epoch}: Loss={loss.item():.6f} | TrainAcc={acc_train:.4f}")

    model.eval()
    with torch.no_grad():
        out = model(data)
        preds = out.argmax(dim=1).cpu().numpy()
        probs = torch.softmax(out, dim=1)[:, 1].cpu().numpy()

    y_test = y_labels[test_idx]
    y_pred_test = preds[test_idx]
    y_pred_test = preds[test_idx]
    y_prob_test = probs[test_idx]

    acc = accuracy_score(y_test, y_pred_test)
    prec = precision_score(y_test, y_pred_test, zero_division=0)
    rec = recall_score(y_test, y_pred_test, zero_division=0)
    f1 = f1_score(y_test, y_pred_test, zero_division=0)
    auc = roc_auc_score(y_test, y_prob_test)
    ce = log_loss(y_test, y_prob_test)

    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1_scores.append(f1)
    aucs.append(auc)
    ce_losses.append(ce)

    print(f"Fold {fold} → "
          f"Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | "
          f"F1={f1:.4f} | AUC={auc:.4f} | CE Loss={ce:.4f}")


# Final summary
print("\n=== Average Results Across 20 Folds ===")
print(f"Accuracy:  {np.mean(accuracies):.4f} \u00b1 {np.std(accuracies):.4f}")
print(f"Precision: {np.mean(precisions):.4f} \u00b1 {np.std(precisions):.4f}")
print(f"Recall:    {np.mean(recalls):.4f} \u00b1 {np.std(recalls):.4f}")
print(f"F1-score:  {np.mean(f1_scores):.4f} \u00b1 {np.std(f1_scores):.4f}")
print(f"AUC:       {np.mean(aucs):.4f} \u00b1 {np.std(aucs):.4f}")
print(f"CE Loss:   {np.mean(ce_losses):.4f} \u00b1 {np.std(ce_losses):.4f}")


=== Fold 1 ===
Fold 1 Epoch 1: Loss=0.742786 | TrainAcc=0.5531
Fold 1 Epoch 500: Loss=0.059521 | TrainAcc=0.9721
Fold 1 Epoch 1000: Loss=0.021893 | TrainAcc=0.9972
Fold 1 Epoch 1500: Loss=0.011881 | TrainAcc=1.0000
Fold 1 Epoch 2000: Loss=0.012379 | TrainAcc=0.9944
Fold 1 → Acc=0.9398 | Prec=0.9573 | Rec=0.9339 | F1=0.9454 | AUC=0.9858 | CE Loss=0.2281

=== Fold 2 ===
Fold 2 Epoch 1: Loss=0.878341 | TrainAcc=0.5140
Fold 2 Epoch 500: Loss=0.061693 | TrainAcc=0.9777
Fold 2 Epoch 1000: Loss=0.030698 | TrainAcc=0.9888
Fold 2 Epoch 1500: Loss=0.015907 | TrainAcc=1.0000
Fold 2 Epoch 2000: Loss=0.015814 | TrainAcc=0.9972
Fold 2 → Acc=0.9358 | Prec=0.9364 | Rec=0.9494 | F1=0.9429 | AUC=0.9830 | CE Loss=0.2675

=== Fold 3 ===
Fold 3 Epoch 1: Loss=0.779951 | TrainAcc=0.4693
Fold 3 Epoch 500: Loss=0.080536 | TrainAcc=0.9749
Fold 3 Epoch 1000: Loss=0.037646 | TrainAcc=0.9944
Fold 3 Epoch 1500: Loss=0.019973 | TrainAcc=0.9972
Fold 3 Epoch 2000: Loss=0.008963 | TrainAcc=1.0000
Fold 3 → Acc=0.9392 |